# ДЗ 6. Двухуровневые модели рекомендаций

*Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

**Задание 2.**

Обучите модель 2-ого уровня, при этом:

- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар

- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker

- Вырос ли precision@5 при использовании двухуровневой модели?

Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [3]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# max 95
# -- train, < 86 -- / -- val1, 86..92 -- / -- val2, 92+ --
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
val1_users = len(set(data_val_lvl_1['user_id']))
new_val1_users = len(set(data_val_lvl_1['user_id']) - set(data_train_lvl_1['user_id']))

val2_users = len(set(data_val_lvl_2['user_id']))
new_val2_users = len(set(data_val_lvl_2['user_id']) - set(data_train_lvl_1['user_id']))

print('В val1 дата сете {} юзеров'.format(val1_users))
print('В val1 дата сете {} новых юзеров'.format(new_val1_users))

print('В val2 дата сете {} юзеров'.format(val2_users))
print('В val2 дата сете {} новых юзеров'.format(new_val2_users))

В val1 дата сете 2154 юзеров
В val1 дата сете 1 новых юзеров
В val2 дата сете 2042 юзеров
В val2 дата сете 1 новых юзеров


In [5]:
train_users = set(data_train_lvl_1['user_id'])
data_val_lvl_1 = data_val_lvl_1[data_val_lvl_1['user_id'].isin(data_train_lvl_1['user_id'])]
data_val_lvl_2 = data_val_lvl_2[data_val_lvl_2['user_id'].isin(data_train_lvl_1['user_id'])]

In [6]:
# sum(data_train_lvl_1['user_id'] == 296)

In [7]:
# list(set(data_train_lvl_1['user_id'])) == 296

In [8]:
# data_train_lvl_1[data_train_lvl_1['user_id']==744]

In [9]:
# data_val_lvl_1[data_val_lvl_1['user_id']==296]

In [10]:
n_items_before = data_train_lvl_1['item_id'].nunique() # куплены 9 недель назад и ранее до фильтрации

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features)

n_items_after = data_train_lvl_1['item_id'].nunique() # куплены 9 недель назад и ранее после фильтрации
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\snetkova\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\snetkova\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\snetkova\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Decreased # items from 83685 to 3095


In [11]:
# data_train_lvl_1[data_train_lvl_1['user_id']==19]

In [12]:
recommender = MainRecommender(data_train_lvl_1)

In [13]:
# for uid in result_lvl_1['user_id']:
#     print(uid)
#     recommender.get_similar_users_recommendation(uid, N=2) # стандартные рекомендации ALS для юзера 2375, топ5
#     print('____')

In [14]:
recommender.get_als_recommendations(296, N=5) # стандартные рекомендации ALS для юзера 2375, топ5

[911878, 6534178, 1007195, 834484, 5978648]

In [15]:
recommender.get_own_recommendations(296, N=5) # стандартные рекомендации из купленных для юзера 2375, топ5

[6534178, 6534178, 5978656, 834484, 866227]

In [16]:
recommender.get_similar_items_recommendation(296, N=5) # похожие на топ5 купленных юзером

[5978648, 6534178, 5978656, 834484, 866227]

In [17]:
recommender.get_similar_users_recommendation(743, N=1) # топ1 купленных похожими 5ю юзерами

[1068651]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [18]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(0)

,user_id,actual


In [19]:
result_lvl_1['als_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=50))
result_lvl_1.head(2)

,user_id,actual,als_recommendation
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[907014, 5978656, 999318, 834484, 866227, 8998..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[903674, 1119879, 1022843, 833715, 1003421, 86..."


In [20]:
result_lvl_1['own_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1.head(2)

,user_id,actual,als_recommendation,own_recommendation
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[907014, 5978656, 999318, 834484, 866227, 8998...","[899838, 1100368, 976300, 866227, 1095275, 102..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[903674, 1119879, 1022843, 833715, 1003421, 86...","[958252, 1039549, 900072, 968215, 866950, 1022..."


In [21]:
result_lvl_1['similar_items_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))
result_lvl_1.head(2)

,user_id,actual,als_recommendation,own_recommendation,similar_items_recommendation
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[907014, 5978656, 999318, 834484, 866227, 8998...","[899838, 1100368, 976300, 866227, 1095275, 102...","[5978648, 5978656, 5978656, 9194758, 857849, 1..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[903674, 1119879, 1022843, 833715, 1003421, 86...","[958252, 1039549, 900072, 968215, 866950, 1022...","[908846, 1003421, 902194, 839753, 917494, 1003..."


In [22]:
result_lvl_1['similar_users_recommendation'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))
result_lvl_1.head(2)

,user_id,actual,als_recommendation,own_recommendation,similar_items_recommendation,similar_users_recommendation
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[907014, 5978656, 999318, 834484, 866227, 8998...","[899838, 1100368, 976300, 866227, 1095275, 102...","[5978648, 5978656, 5978656, 9194758, 857849, 1...","[7440663, 1027960, 1092948, 857849, 907014, 11..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[903674, 1119879, 1022843, 833715, 1003421, 86...","[958252, 1039549, 900072, 968215, 866950, 1022...","[908846, 1003421, 902194, 839753, 917494, 1003...","[958252, 976300, 958252, 1060567, 6534166, 866..."


In [23]:
result_lvl_1.apply(lambda row: precision_at_k(row['als_recommendation'], row['actual']), axis=1).mean()

0.07505805852299113

In [24]:
result_lvl_1.apply(lambda row: precision_at_k(row['own_recommendation'], row['actual']), axis=1).mean()

0.13478866697631103

In [25]:
result_lvl_1.apply(lambda row: precision_at_k(row['similar_items_recommendation'], row['actual']), axis=1).mean()

0.08490478402229423

In [26]:
result_lvl_1.apply(lambda row: precision_at_k(row['similar_users_recommendation'], row['actual']), axis=1).mean()

0.03241987923827232

own recommendations + top-popular действительно дают лучший recall

#### best k search

In [27]:
res = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
res.columns=['user_id', 'actual']
res.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [28]:
metrics = {}
for k in [3, 5, 10, 20, 50, 100, 200, 500]:
    res['own' + str(k)] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=k))
    metrics['own' + str(k)] = res.apply(lambda row: recall_at_k(row['own' + str(k)], row['actual'], k), axis=1).mean()
res.head()
metrics

{'own3': 0.010041738112825086,
 'own5': 0.013136072429192737,
 'own10': 0.020605685120485478,
 'own20': 0.029201744285348417,
 'own50': 0.04317170452301033,
 'own100': 0.052249545042563385,
 'own200': 0.05986570376143409,
 'own500': 0.07013749256485244}

С увеличением количества рекомендуемых товаров метрика recall@k растет

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [29]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))

In [30]:
users_lvl_2.head(2)

,user_id,candidates
0,2070,"[878302, 1055863, 15452263, 949616, 1080414, 1..."
1,2021,"[910291, 1092948, 1019744, 1029195, 1122268, 1..."


In [31]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,878302,1
0,2070,1055863,1
0,2070,15452263,1
0,2070,949616,1


In [32]:
users_lvl_2.shape[0], users_lvl_2['user_id'].nunique()

(107650, 2153)

In [33]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [34]:
targets_lvl_2.head(10)

,user_id,item_id,target
0,2070,878302,1.0
1,2070,878302,1.0
2,2070,878302,1.0
3,2070,878302,1.0
4,2070,878302,1.0
5,2070,878302,1.0
6,2070,1055863,1.0
7,2070,15452263,0.0
8,2070,949616,1.0
9,2070,949616,1.0


In [35]:
targets_lvl_2['target'].mean()

0.0878633956554631

#### фичи

In [36]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [37]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [38]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [39]:
user_stats = data.groupby('user_id')['quantity', 'sales_value', 'week_no', 'basket_id'].agg(money = ('sales_value', 'sum'), qty = ('quantity', 'sum'),  bask = ('basket_id', 'nunique'), wk = ('week_no', 'nunique')).reset_index()
user_stats['avg_bill'] = user_stats['money'] / user_stats['bask']
user_stats['weekly_purch'] = user_stats['bask'] / user_stats['wk']
user_stats.head()

C:\Users\snetkova\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,user_id,money,qty,bask,wk,avg_bill,weekly_purch
0,1,3959.91,1844,79,64,50.125443,1.234375
1,2,1823.45,763,44,34,41.442045,1.294118
2,3,2594.30,8503,45,35,57.651111,1.285714
3,4,1200.11,382,30,26,40.003667,1.153846
4,5,749.09,242,38,25,19.712895,1.520000


In [40]:
user_features = user_features.merge(user_stats[['user_id', 'avg_bill', 'weekly_purch']], how = 'left', on = 'user_id')

In [41]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801 entries, 0 to 800
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age_desc             801 non-null    object 
 1   marital_status_code  801 non-null    object 
 2   income_desc          801 non-null    object 
 3   homeowner_desc       801 non-null    object 
 4   hh_comp_desc         801 non-null    object 
 5   household_size_desc  801 non-null    object 
 6   kid_category_desc    801 non-null    object 
 7   user_id              801 non-null    int64  
 8   avg_bill             801 non-null    float64
 9   weekly_purch         801 non-null    float64
dtypes: float64(2), int64(1), object(7)
memory usage: 68.8+ KB


In [42]:
item_stats = data.groupby('item_id')['quantity', 'sales_value', 'week_no'].agg(money = ('sales_value', 'sum'), qty = ('quantity', 'sum'), wk = ('week_no', 'nunique')).reset_index()
item_stats['price'] = item_stats['money'] / (item_stats['qty']+1e5)
item_stats['weekly'] = item_stats['qty'] / (item_stats['wk']+1e5)
item_stats.head()

C:\Users\snetkova\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,item_id,money,qty,wk,price,weekly
0,25671,20.94,6,3,0.000209,0.00006
1,26081,0.99,1,1,0.000010,0.00001
2,26093,1.59,1,1,0.000016,0.00001
3,26190,1.54,1,1,0.000015,0.00001
4,26355,1.98,2,1,0.000020,0.00002


In [43]:
item_features = item_features.merge(item_stats[['item_id', 'price', 'weekly']], how = 'left', on = 'item_id')

In [44]:
item_features = item_features.fillna(0)

In [45]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92353 entries, 0 to 92352
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   item_id               92353 non-null  int64  
 1   manufacturer          92353 non-null  int64  
 2   department            92353 non-null  object 
 3   brand                 92353 non-null  object 
 4   commodity_desc        92353 non-null  object 
 5   sub_commodity_desc    92353 non-null  object 
 6   curr_size_of_product  92353 non-null  object 
 7   price                 92353 non-null  float64
 8   weekly                92353 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 7.0+ MB


In [46]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price,weekly
0,2070,878302,1.0,544,GROCERY,National,BAG SNACKS,SGL SV/VEND MACH SNACKS CHIP/P,1 OZ,0.00239,0.008033
1,2070,878302,1.0,544,GROCERY,National,BAG SNACKS,SGL SV/VEND MACH SNACKS CHIP/P,1 OZ,0.00239,0.008033


In [47]:
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,price,weekly,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,avg_bill,weekly_purch
0,2070,878302,1.0,544,GROCERY,National,BAG SNACKS,SGL SV/VEND MACH SNACKS CHIP/P,1 OZ,0.00239,0.008033,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,12.92937,6.585366
1,2070,878302,1.0,544,GROCERY,National,BAG SNACKS,SGL SV/VEND MACH SNACKS CHIP/P,1 OZ,0.00239,0.008033,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,12.92937,6.585366


In [48]:
targets_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111036 entries, 0 to 111035
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   user_id               111036 non-null  int64  
 1   item_id               111036 non-null  int64  
 2   target                111036 non-null  float64
 3   manufacturer          111036 non-null  int64  
 4   department            111036 non-null  object 
 5   brand                 111036 non-null  object 
 6   commodity_desc        111036 non-null  object 
 7   sub_commodity_desc    111036 non-null  object 
 8   curr_size_of_product  111036 non-null  object 
 9   price                 111036 non-null  float64
 10  weekly                111036 non-null  float64
 11  age_desc              41685 non-null   object 
 12  marital_status_code   41685 non-null   object 
 13  income_desc           41685 non-null   object 
 14  homeowner_desc        41685 non-null   object 
 15  

In [49]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [50]:
X_train.columns

Index(['user_id', 'item_id', 'manufacturer', 'department', 'brand',
       'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product', 'price',
       'weekly', 'age_desc', 'marital_status_code', 'income_desc',
       'homeowner_desc', 'hh_comp_desc', 'household_size_desc',
       'kid_category_desc', 'avg_bill', 'weekly_purch'],
      dtype='object')

In [51]:
cat_feats = X_train.select_dtypes(include='object').columns.tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [52]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)[:, 1]

C:\Users\snetkova\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\snetkova\Anaconda3\lib\site-packages\lightgbm\basic.py:814: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


In [53]:
probs = pd.Series(train_preds, name = 'probs', index = X_train.index)
probs

0         0.523085
1         0.523085
2         0.523085
3         0.523085
4         0.523085
            ...   
111031    0.003600
111032    0.036320
111033    0.045492
111034    0.042602
111035    0.036302
Name: probs, Length: 111036, dtype: float64

In [54]:
item_probs = pd.concat([X_train[['user_id', 'item_id']], probs], axis = 1).drop_duplicates()
item_probs.head()

,user_id,item_id,probs
0,2070,878302,0.523085
6,2070,1055863,0.198914
7,2070,15452263,0.029512
8,2070,949616,0.628520
18,2070,1080414,0.263191


In [55]:
item_probs = item_probs.sort_values(by = ['user_id', 'probs'], ascending = False)
item_probs.head(10)

,user_id,item_id,probs
71825,2500,15595945,0.227455
71796,2500,951703,0.199052
71803,2500,15596094,0.196911
71802,2500,825618,0.193206
71847,2500,976955,0.115827
71805,2500,1088393,0.094491
71808,2500,1090017,0.094491
71822,2500,5584007,0.068681
71818,2500,5589247,0.066593
71836,2500,5584027,0.062990


In [56]:
item_probs['row_num'] = item_probs.sort_values(['probs'], ascending = False).groupby(['user_id']).cumcount()+1

In [57]:
item_probs.groupby(['user_id'])['item_id', 'probs'].rank(method = 'first')

C:\Users\snetkova\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,item_id,probs
71825,48.0,50.0
71796,17.0,49.0
71803,49.0,48.0
71802,1.0,47.0
71847,22.0,46.0
...,...,...
46222,35.0,5.0
46237,28.0,4.0
46201,32.0,3.0
46203,44.0,2.0


In [58]:
item_probs = item_probs.loc[item_probs['row_num']<=10]

In [59]:
item_probs

,user_id,item_id,probs,row_num
71825,2500,15595945,0.227455,1
71796,2500,951703,0.199052,2
71803,2500,15596094,0.196911,3
71802,2500,825618,0.193206,4
71847,2500,976955,0.115827,5
...,...,...,...,...
46227,1,6533889,0.110233,7
46232,1,6534166,0.110233,6
46220,1,1013321,0.108681,8
46224,1,976998,0.106476,9


In [60]:
lgbm_recs = item_probs.groupby('user_id')['item_id'].agg(lgbm_recs = 'unique').reset_index()
lgbm_recs.head()

,user_id,lgbm_recs
0,1,"[5978656, 5978648, 6534178, 834484, 820165, 65..."
1,2,"[6534178, 5978656, 5978648, 962118, 834484, 95..."
2,4,"[6534178, 5978656, 5978648, 834484, 9526563, 8..."
3,6,"[6534178, 1404121, 939578, 5978648, 5978656, 8..."
4,7,"[5978656, 5978648, 836281, 1013321, 13876341, ..."


In [61]:
result_lvl_1 = result_lvl_1.merge(lgbm_recs, how = 'left', on = 'user_id')

### result compare

In [62]:
result_lvl_1.apply(lambda row: precision_at_k(row['own_recommendation'], row['actual']), axis=1).mean()

0.13478866697631103

In [63]:
result_lvl_1.apply(lambda row: precision_at_k(row['lgbm_recs'], row['actual']), axis=1).mean()

0.1834649326521114

После второго уровна precision ощутимо вырос

### Финальный проект

Мы уже прошли всю необходимую теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 